In [1]:
%matplotlib inline
import MySQLdb as mdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# Helper Function 

In [2]:
def fetch(cols,table,where=None,group_by=None,order_by=None,Desc=False,limit=None):
    """
    Purpose: Makes the MySQL quesries more efficitent and faster.
    
    Note: All inputs have to be in STRING format except the 'limit' which is an int
    """
    
    # Opening a Connection to donorsChoose Database
    con = mdb.connect('localhost', 'idx', 'donorsChoose', 'donors');
    cur = con.cursor()
    
    # Creating a query
    query = 'SELECT ' + cols + ' FROM ' + table + ' '
    if where: query += ' WHERE ' + where + ' '
    if group_by: query += 'GROUP BY ' + group_by + ' '
    if order_by: query += 'ORDER BY ' + order_by + ' '
    if Desc: query += 'DESC '
    if limit: query += 'LIMIT ' + str(limit)
    
    # fetching the data and turn it into a pandas dataframe
    cur.execute(query)
    output = np.array(cur.fetchall())
    colm = [f[0] for f in cur.description]
    output = pd.DataFrame(output)
    output.columns = colm

    # Closing the connection to the database
    con.close()
    
    return output

## Finding The Donation Made to LA Projects 

Project ID is a common key between donation and project tables. So, let's find the projects in LA. Then, we can find all the donations made to the LA projects.

This can be accomplished with a simple inner join between funding status and la_donation.

This part is done in MySQL. Here is the code:

In [16]:
la_donations = fetch('*','donation_LA')

In [17]:
la_donations.head()

,_projectid,_donationid,is_teacher_acct,payment_method,payment_included_acct_credit,payment_included_campaign_gift_card,payment_included_web_purchased_gift_card,donation_total,dollar_amount,payment_was_promo_matched,via_giving_page,for_honoree,donor_city,donation_message,funding_status
0,"""""""0434af307dc45294c2eb748ffd935767""""""","""""""83f29a83b88d0c7a1c96cf9bd9f3b2ab""""""",0,no_cash_received,1,0,0,75.00,10_to_100,f,0,0,,,completed
1,"""""""59f17c3f118cef16b2fee23ed74753fe""""""","""""""db4df4b7ed7f3e2145fc4550243304d9""""""",0,creditcard,0,0,0,100.00,100_and_up,f,0,0,LOS ANGELES,"""""""I gave to this project because I studied ab...",completed
2,"""""""464e9eb3a3918a2d3f2006513bd42949""""""","""""""36a2ca9827e7d6ea6eb6235205b55309""""""",0,creditcard,0,0,0,40.00,10_to_100,f,0,0,"""""","""""""readers are like super heroes",completed
3,"""""""183af384da17ccbdd72140d8daabeb8a""""""","""""""c71d10db81e2c2e455d5a62dcf87550b""""""",0,no_cash_received,0,1,0,10.00,10_to_100,f,0,0,"""""","""""""GOD BLESS YOU! KEEP UP THE GOOD WORK!""""""",completed
4,"""""""350299a9765c77228f2bcb2c450a095d""""""","""""""8a8b40c32e06554570241cec6295e80c""""""",0,no_cash_received,0,1,0,25.00,10_to_100,f,0,0,,,completed


## Feature Engineering 

Okay, so what are the potential features?

In [19]:
la_donations['funding_status_binary'] = la_donations.funding_status.map( {'completed': 1, 'expired': 0}).astype(int)

In [22]:
sum(la_donations.is_teacher_acct.isnull())

0

In [23]:
la_donations.groupby(['payment_method'])['funding_status_binary'].apply(\
                            lambda x: (np.mean(x) - 1.96*np.sqrt(np.mean(x)*(1-np.mean(x))/len(x)),\
                                       np.mean(x) + 1.96*np.sqrt(np.mean(x)*(1-np.mean(x))/len(x))))

payment_method
almost_home_match                                 (1.0, 1.0)
amazon                      (0.913889034612, 0.930401622828)
check                       (0.961322741381, 0.984273113541)
creditcard                   (0.93678982861, 0.941695352509)
double_your_impact_match                          (1.0, 1.0)
no_cash_received            (0.934958208129, 0.937886296294)
paypal                      (0.925519663773, 0.935521520397)
promo_code_match            (0.945791924635, 0.952876915354)
Name: funding_status_binary, dtype: object

In [216]:
projects_donations_la.columns

Index([u'_projectid', u'_teacher_acctid', u'_schoolid', u'school_ncesid',
       u'school_latitude', u'school_longitude', u'school_city',
       u'school_state', u'school_zip', u'school_metro', u'school_district',
       u'school_county', u'school_charter', u'school_magnet',
       u'school_year_round', u'school_nlns', u'school_kipp',
       u'school_charter_ready_promise', u'teacher_prefix',
       u'teacher_teach_for_america', u'teacher_ny_teaching_fellow',
       u'primary_focus_subject', u'primary_focus_area',
       u'secondary_focus_subject', u'secondary_focus_area', u'resource_type',
       u'poverty_level', u'grade_level', u'vendor_shipping_charges',
       u'sales_tax', u'payment_processing_charges',
       u'fulfillment_labor_materials',
       u'total_price_excluding_optional_support',
       u'total_price_including_optional_support', u'students_reached',
       u'total_donations', u'num_donors', u'eligible_double_your_impact_match',
       u'eligible_almost_home_match', u'f

In [249]:
projects_donations_la.groupby('school_zip').total_donations.mean()

school_zip
9001      458.770000
90001     661.894320
90002     584.937177
90003     680.793888
90004     580.458395
90005     850.052593
90006     576.762166
90007     791.229370
90008    1161.291051
90010     866.726400
90011    1133.246199
90012     725.466021
90015     708.179067
90016     802.557543
90017    1988.388755
90018     624.118355
90019     634.930454
90021     546.029734
90022     602.736738
90023     583.806900
90024    3290.116037
90025     666.896968
90026     696.597304
90027     534.366420
90028     819.729051
90029     711.389765
90031     549.343615
90032     669.409618
90033     589.494465
90034     585.718115
            ...     
91750     223.707288
91754     728.653564
91755       0.000000
91765     605.685041
91766     444.235455
91767     357.299215
91768     657.289154
91770     593.036584
91773     685.500681
91775     587.913556
91776     791.290973
91780    1060.341362
91789     849.433333
91790    1013.849309
91791     440.952174
91792     399.966564
91

In [251]:
projects_donations_la.groupby('school_zip').num_donors.mean()

school_zip
9001     12.583333
90001    20.281827
90002    16.029231
90003    19.646871
90004    14.978911
90005    22.897305
90006    21.200358
90007    18.908572
90008    13.800930
90010     8.520000
90011    24.956159
90012    23.374522
90015    19.007131
90016    24.831281
90017    24.703975
90018    16.282940
90019    16.970936
90021     9.355482
90022    17.841793
90023    16.375624
90024    23.208166
90025    14.884425
90026    20.172777
90027    15.769748
90028    19.746237
90029    27.048387
90031    15.564717
90032    19.273789
90033    17.131801
90034    20.010783
           ...    
91750     7.118644
91754    10.405941
91755     1.666667
91765     9.867769
91766     5.925620
91767     6.780105
91768    20.146154
91770    11.334158
91773    66.126783
91775    38.111925
91776    32.250450
91780    16.103286
91789     7.928571
91790    13.321291
91791     6.423913
91792    14.405128
91801    11.436090
91803     7.923077
93243     1.000000
93510    14.557522
93532     7.000000
9

1. Does the success rate vary by zipcode?
2. how many unique donors at each zipcode
3. grouping zipcode --> how?

Donations made by the zip, city, state of the project:
a. sum and mean amount donated